# Data Creation Process

In [1]:
# import requirements
import pandas as pd
import numpy as np

## Sources

[NY County Level Test Data](https://health.data.ny.gov/Health/New-York-State-Statewide-COVID-19-Testing/xdss-u53e/data)

[County Level Economic Variables](https://raw.githubusercontent.com/JieYingWu/COVID-19_US_County-level_Summaries/master/data/counties.csv)

[NY Municipality Codes](https://data.ny.gov/Government-Finance/NY-Municipalities-and-County-FIPS-codes/79vr-2kdi)


In [2]:
ny_data = pd.read_csv("./raw_data/New_York_State_Statewide_COVID-19_Testing.csv")
county_codes = pd.read_csv("./raw_data/NY_Municipalities_and_County_FIPS_codes.csv")
county_data = pd.read_csv("https://raw.githubusercontent.com/JieYingWu/COVID-19_US_County-level_Summaries/master/data/counties.csv")

## Creating County Level Testing and Economic Data 

In [3]:
ny_data.head()

,Test Date,County,New Positives,Cumulative Number of Positives,Total Number of Tests Performed,Cumulative Number of Tests Performed
0,05/13/2020,Albany,15,1509,546,15792
1,05/13/2020,Allegany,0,44,53,938
2,05/13/2020,Bronx,303,41980,3338,127471
3,05/13/2020,Broome,1,383,343,6146
4,05/13/2020,Cattaraugus,3,64,134,1708


#### To avoid problems due to differences in how the county names are written, we want to use FIPS (county code) to identify counties. Since the NY site does not have the county codes, we will use a dataset that maps  county names to FIPS and join the two datasets.

In [4]:
#correcting a naming difference between the datasets
county_codes.replace("St Lawrence","St. Lawrence",inplace=True)

In [5]:
county_code_df = county_codes.groupby(['County Name','County FIPS']).size().reset_index().drop(0,axis=1)

In [6]:
#joining ny data w codes data to get FIPS info
ny_data_fips = ny_data.set_index("County").join(county_code_df.set_index("County Name")).astype({'County FIPS': 'int32'})

#### Collecting a list of columns that have the relevant infection and socioeconomic data we want to explore

In [7]:
relevant_ses_columns = ["Median_Household_Income_2018",\
                        "transit_scores - population weighted averages aggregated from town/city level to county"]
relevant_infection_columns = ['Test Date','New Positives',\
                              'Cumulative Number of Positives',\
                              'Total Number of Tests Performed','Cumulative Number of Tests Performed',]

#### Joining testing data with socioeconomic data

In [23]:
#joining infection w socioeconomic
joined_df = county_data[county_data['State']=='NY'\
                               ].set_index('FIPS').join(ny_data_fips.set_index('County FIPS'))
#dropping null column
joined_df = joined_df[relevant_ses_columns\
                                      +relevant_infection_columns].drop(36000)

In [24]:
#doing some clean up
joined_df = joined_df.reset_index().rename(columns={'index':'FIPS'})
joined_df['Test Date'] = pd.to_datetime(joined_df['Test Date'])

In [25]:
joined_df.head()

,FIPS,Median_Household_Income_2018,transit_scores - population weighted averages aggregated from town/city level to county,Test Date,New Positives,Cumulative Number of Positives,Total Number of Tests Performed,Cumulative Number of Tests Performed
0,36001,64536.0,7.100041e+09,2020-05-13,15.0,1509.0,546.0,15792.0
1,36001,64536.0,7.100041e+09,2020-05-12,15.0,1494.0,335.0,15246.0
2,36001,64536.0,7.100041e+09,2020-05-11,23.0,1479.0,341.0,14911.0
3,36001,64536.0,7.100041e+09,2020-05-10,24.0,1456.0,384.0,14570.0
4,36001,64536.0,7.100041e+09,2020-05-09,16.0,1432.0,455.0,14186.0


In [31]:
joined_df.to_csv("./ny_county_data/ny_county_data.csv")